# ResNet 2개층만 미세조정 모델 ( model3)

In [7]:
# Pre-trained ResNet50 모델을 불러옵니다.
base_model_2 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) #최상위 분류 레이어를 제외하고, 기본적인 특성 추출 부분만 불러옵니다.
base_model_2.trainable = False           # 모든 레이어를 동결
for layer in base_model_2.layers[-2:]:  # 마지막 2개 레이어만 동결 해제
    layer.trainable = True

94781440/94765736 [==============================] - 0s 0us/step


In [8]:
x = base_model_2.output
x = GlobalAveragePooling2D()(x)    # 경량화와 빠른 학습, 적은 파라미터로 처리
#x = Flatten()(base_model.output)  # 많은 파라미터 계산, 고급 모델에서 정밀한 분류시 유리
x = Dense(1024, activation='relu')(x)
outputs = Dense(6, activation='softmax')(x) # 해파리를 6개의 클래스로 분류
model_2 = Model(base_model_2.input, outputs)
model_2.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [9]:
model_2.compile(
    optimizer='adam',   # Adam 최적화 알고리즘 (일반적으로 잘 동작)
    loss='sparse_categorical_crossentropy',  # 타겟의 출력이 원-핫 인코딩된 형태가 아니라 정수형 라벨로 되어 있음. 다중 클래스 분류를 위한 손실 함수
    metrics=['accuracy']  # 모델의 정확도를 평가 지표로 사용
)

In [10]:
callbacks = [
    
    keras.callbacks.ModelCheckpoint(
    'ResNet_model.keras_3',  # 모델 저장 경로
    monitor='val_loss',    # 검증 손실을 기준으로 저장
    save_best_only=True,  # 가장 좋은 모델만 저장
    verbose=1)             # 저장 시 메시지 출력
]

history = model_2.fit(
    train_ds,  # 훈련 데이터
    epochs=100,
    validation_data=val_ds,  # 검증 데이터
    callbacks=callbacks)

Epoch 1/100
43/43 [==============================] - 45s 105ms/step - loss: 2.1138 - accuracy: 0.1650 - val_loss: 1.9165 - val_accuracy: 0.1633

Epoch 00001: val_loss improved from inf to 1.91648, saving model to ResNet_model.keras_3
INFO:tensorflow:Assets written to: ResNet_model.keras_3/assets
Epoch 2/100
43/43 [==============================] - 6s 64ms/step - loss: 2.0036 - accuracy: 0.1752 - val_loss: 1.9319 - val_accuracy: 0.1990

Epoch 00002: val_loss did not improve from 1.91648
Epoch 3/100
43/43 [==============================] - 6s 64ms/step - loss: 1.8559 - accuracy: 0.1664 - val_loss: 1.8302 - val_accuracy: 0.2245

Epoch 00003: val_loss improved from 1.91648 to 1.83023, saving model to ResNet_model.keras_3
INFO:tensorflow:Assets written to: ResNet_model.keras_3/assets
Epoch 4/100
43/43 [==============================] - 6s 64ms/step - loss: 1.8011 - accuracy: 0.1971 - val_loss: 1.7717 - val_accuracy: 0.1939

Epoch 00004: val_loss improved from 1.83023 to 1.77167, saving mode

In [18]:
test_loss, test_accuracy = model_2.evaluate(test_ds, verbose=2)

# 평가 결과 출력
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

7/7 - 1s - loss: 1.5335 - accuracy: 0.3469
Test Loss: 1.5335311889648438
Test Accuracy: 0.3469387888908386


In [20]:
model3 = tf.keras.models.load_model('ResNet_model.keras_3')

test_loss, test_accuracy = model3.evaluate(test_ds, verbose=2)

# 평가 결과 출력
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

7/7 - 2s - loss: 1.5300 - accuracy: 0.4184
Test Loss: 1.5299738645553589
Test Accuracy: 0.4183673560619354
